<a href="https://colab.research.google.com/github/M-110/testing-with-pytest/blob/main/04_Builtin_Fixtures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Highlights

* tmpdir
* create file as a fixture
* add options to parser
* cache
* capsys capture stdout/errors
* doc test tests

# Using tmpdir and tmpdir_factory

In [ ]:
%%writefile test_tmpdir.py
import pytest


def test_tmpdir(tmpdir):
  my_file = tmpdir.join('something.txt')
  my_dir = tmpdir.mkdir('some_dir')
  another_file = my_dir.join('something_else.txt')
  
  my_file.write('this is my first file')
  
  another_file.write('this is my second file.')
  
  assert 'first' in my_file.read()
  assert 'first' in another_file.read()

Writing test_tmpdir.py


In [ ]:
!pytest test_tmpdir.py --setup-show -v

============================= test session starts ==============================
platform linux2 -- Python 2.7.17, pytest-3.6.4, py-1.8.0, pluggy-0.7.1 -- /usr/bin/python2
cachedir: .pytest_cache
rootdir: /content, inifile:
collected 1 item                                                               

test_tmpdir.py::test_tmpdir 
SETUP    S doctest_namespace
SETUP    S tmpdir_factory
      SETUP    F add_mm (fixtures used: doctest_namespace)
      SETUP    F tmpdir (fixtures used: tmpdir_factory)
        test_tmpdir.py::test_tmpdir (fixtures used: add_mm, doctest_namespace, tmpdir, tmpdir_factory)FAILED
      TEARDOWN F tmpdir
      TEARDOWN F add_mm
TEARDOWN S tmpdir_factory
TEARDOWN S doctest_namespace

=================================== FAILURES ===================================
_________________________________ test_tmpdir __________________________________

tmpdir = local('/tmp/pytest-of-root/pytest-0/test_tmpdir0')

    def test_tmpdir(tmpdir):
      my_file = tmpdir.join('s

In [ ]:
%%writefile test_tmpdir2.py
import pytest

def test_tmpdir_factory(tmpdir_factory):
  a_dir = tmpdir_factory.mktemp('mydir')
  base_temp = tmpdir_factory.getbasetemp()
  print('base:', base_temp)
  my_file = a_dir.join('something.txt')
  my_dir = a_dir.mkdir('some_dir')
  another_file = my_dir.join('something_else.txt')
  
  my_file.write('this is my first file')
  
  another_file.write('this is my second file.')
  
  assert 'first' in my_file.read()
  assert 'second' in another_file.read()

Writing test_tmpdir2.py


In [ ]:
!pytest test_tmpdir2.py -s --setup-show

============================= test session starts ==============================
platform linux2 -- Python 2.7.17, pytest-3.6.4, py-1.8.0, pluggy-0.7.1
rootdir: /content, inifile:
collected 1 item                                                               

test_tmpdir2.py 
SETUP    S tmpdir_factory
SETUP    S doctest_namespace
      SETUP    F add_mm (fixtures used: doctest_namespace)
        test_tmpdir2.py::test_tmpdir_factory (fixtures used: add_mm, doctest_namespace, tmpdir_factory)('base:', local('/tmp/pytest-of-root/pytest-1'))
.
      TEARDOWN F add_mm
TEARDOWN S doctest_namespace
TEARDOWN S tmpdir_factory

=========================== 1 passed in 0.02 seconds ===========================


In [ ]:
%%writefile conftest.py
import json
import pytest

@pytest.fixture(scope='module')
def author_file_json(tmpdir_factory):
  python_author_data = dict(Ned=dict(City='Boston'),
                            Brian=dict(City='Portland'),
                            Luciano=dict(City='Sao Paulo'))
  file = tmpdir_factory.mktemp('data').join('author_file.json')
  print('file: ', str(file))

  with file.open('w') as f:
    json.dump(python_author_data, f)
  return file

Overwriting conftest.py


In [ ]:
%%writefile test_authors.py
import json

def test_brian(author_file_json):
  with author_file_json.open() as f:
    authors = json.load(f)
  assert authors['Brian']['City'] == 'Portland'

def test_all_have_cities(author_file_json):
  with author_file_json.open() as f:
    authors = json.load(f)
  for a in authors:
    assert len(authors[a]['City']) > 0
  


Writing test_authors.py


In [ ]:
!pytest test_authors.py --setup-show -s -v

============================= test session starts ==============================
platform linux2 -- Python 2.7.17, pytest-3.6.4, py-1.8.0, pluggy-0.7.1 -- /usr/bin/python2
cachedir: .pytest_cache
rootdir: /content, inifile:
collected 2 items                                                              

test_authors.py::test_brian 
SETUP    S tmpdir_factory('file: ', '/tmp/pytest-of-root/pytest-2/data0/author_file.json')

  SETUP    M author_file_json (fixtures used: tmpdir_factory)
        test_authors.py::test_brian (fixtures used: author_file_json, tmpdir_factory)PASSED
test_authors.py::test_all_have_cities 
        test_authors.py::test_all_have_cities (fixtures used: author_file_json, tmpdir_factory)PASSED
  TEARDOWN M author_file_json
TEARDOWN S tmpdir_factory

=========================== 2 passed in 0.02 seconds ===========================


# Using pytest config

In [ ]:
%%writefile conftest.py
def pytest_addoption(parser):
  parser.addoption('--myopt', action='store_true', help='some bool option')
  parser.addoption('--pet', action='store', default='cat', help='pet: cat or dog')

Overwriting conftest.py


In [ ]:
!pytest --help

usage: pytest [options] [file_or_dir] [file_or_dir] [...]

positional arguments:
  file_or_dir

general:
  -k EXPRESSION         only run tests which match the given substring
                        expression. An expression is a python evaluatable
                        expression where all names are substring-matched
                        against test names and their parent classes. Example:
                        -k 'test_method or test_other' matches all test
                        functions and classes whose name contains
                        'test_method' or 'test_other', while -k 'not
                        test_method' matches those that don't contain
                        'test_method' in their names. Additionally keywords
                        are matched to classes and functions containing extra
                        names in their 'extra_keyword_matches' set, as well as
                        functions which have names assigned directly to them.
  -m MARKEX

In [ ]:
%%writefile test_config.py

import pytest


def test_option(pytestconfig):
  print('your pet is', pytestconfig.getoption('pet'))
  print('your myopt is', pytestconfig.getoption('myopt'))

Writing test_config.py


In [ ]:
!pytest test_config.py --pet dog --myopt -s -v

============================= test session starts ==============================
platform linux2 -- Python 2.7.17, pytest-3.6.4, py-1.8.0, pluggy-0.7.1 -- /usr/bin/python2
cachedir: .pytest_cache
rootdir: /content, inifile:
collected 1 item                                                               

test_config.py::test_option ('your pet is', 'dog')
('your myopt is', True)
PASSED

=========================== 1 passed in 0.01 seconds ===========================


In [ ]:
!pytest test_config.py  -s -v

============================= test session starts ==============================
platform linux2 -- Python 2.7.17, pytest-3.6.4, py-1.8.0, pluggy-0.7.1 -- /usr/bin/python2
cachedir: .pytest_cache
rootdir: /content, inifile:
collected 1 item                                                               

test_config.py::test_option ('your pet is', 'cat')
('your myopt is', False)
PASSED

=========================== 1 passed in 0.01 seconds ===========================


In [ ]:
%%writefile test_config2.py

import pytest

@pytest.fixture()
def pet(pytestconfig):
  return pytestconfig.option.pet

@pytest.fixture()
def myopt(pytestconfig):
  return pytestconfig.option.myopt

def test_fixtures_for_options(pet, myopt):
  print('my pet is', pet)
  print('my opt is ', myopt)

Writing test_config2.py


In [ ]:
!pytest test_config2.py -s

============================= test session starts ==============================
platform linux2 -- Python 2.7.17, pytest-3.6.4, py-1.8.0, pluggy-0.7.1
rootdir: /content, inifile:
collected 1 item                                                               

test_config2.py ('my pet is', 'cat')
('my opt is ', False)
.

=========================== 1 passed in 0.01 seconds ===========================


In [ ]:
%%writefile test_config3.py 

def test_pytestconfig(pytestconfig):
  print('args            :', pytestconfig.args)
  print('inifile         :', pytestconfig.inifile)
  print('invocation_dir  :', pytestconfig.invocation_dir)
  print('rootdir         :', pytestconfig.rootdir)
  print('-k EXPRESSION   :', pytestconfig.getoption('keyword'))
  print('-v, --verbose   :', pytestconfig.getoption('verbose'))
  print('-l, --showlocals:', pytestconfig.getoption('showlocals'))
  print('--tb=style      :', pytestconfig.getoption('tbstyle'))

Writing test_config3.py


In [ ]:
!pytest test_config3.py -v -s -l

============================= test session starts ==============================
platform linux2 -- Python 2.7.17, pytest-3.6.4, py-1.8.0, pluggy-0.7.1 -- /usr/bin/python2
cachedir: .pytest_cache
rootdir: /content, inifile:
collected 1 item                                                               

test_config3.py::test_pytestconfig ('args            :', ['test_config3.py'])
('inifile         :', None)
('invocation_dir  :', local('/content'))
('rootdir         :', local('/content'))
('-k EXPRESSION   :', '')
('-v, --verbose   :', 1)
('-l, --showlocals:', True)
('--tb=style      :', 'auto')
PASSED

=========================== 1 passed in 0.01 seconds ===========================


# Cache

In [ ]:
!pytest --help | grep 'last-failed\|failed-first\|cache'

  --lf, --last-failed   rerun only the tests that failed at the last run (or
  --ff, --failed-first  run all tests but run the last failures first. This
  --cache-show          show cache contents, don't perform collection or tests
  --cache-clear         remove all cache contents at start of test run.
  --lfnf={all,none}, --last-failed-no-failures={all,none}
                        found in the cache
                        `-o xfail_strict=True -o cache_dir=cache`.
  cache_dir (string)       cache directory path.


In [ ]:
%%writefile test_pass_fail.py

def test_that_passes():
  assert 1 == 1

def test_that_fails():
  assert 1 == 2
  

Writing test_pass_fail.py


In [ ]:
!pytest test_pass_fail.py -v --tb=no

============================= test session starts ==============================
platform linux2 -- Python 2.7.17, pytest-3.6.4, py-1.8.0, pluggy-0.7.1 -- /usr/bin/python2
cachedir: .pytest_cache
rootdir: /content, inifile:
collected 2 items                                                              

test_pass_fail.py::test_that_passes PASSED                               [ 50%]
test_pass_fail.py::test_that_fails FAILED                                [100%]

====================== 1 failed, 1 passed in 0.03 seconds ======================


The tests are cached so that when "ff" (failed first) is called, it remembers which one failed first.

In [ ]:
!pytest test_pass_fail.py -v --tb=no --ff

============================= test session starts ==============================
platform linux2 -- Python 2.7.17, pytest-3.6.4, py-1.8.0, pluggy-0.7.1 -- /usr/bin/python2
cachedir: .pytest_cache
rootdir: /content, inifile:
collected 2 items                                                              
run-last-failure: rerun previous 1 failure first

test_pass_fail.py::test_that_fails FAILED                                [ 50%]
test_pass_fail.py::test_that_passes PASSED                               [100%]

====================== 1 failed, 1 passed in 0.02 seconds ======================


In [ ]:
!pytest test_pass_fail.py -v --tb=no --last-failed

============================= test session starts ==============================
platform linux2 -- Python 2.7.17, pytest-3.6.4, py-1.8.0, pluggy-0.7.1 -- /usr/bin/python2
cachedir: .pytest_cache
rootdir: /content, inifile:
collected 2 items / 1 deselected                                               
run-last-failure: rerun previous 1 failure

test_pass_fail.py::test_that_fails FAILED                                [100%]

==================== 1 failed, 1 deselected in 0.02 seconds ====================


In [ ]:
%%writefile test_some_failures.py

import pytest
from pytest import approx

test_data = [(1.01, 2.01, 3.02),
             (1e25, 1e23, 1.1e25),
             (1.23, 3.21, 4.44),
             (0.1, 0.2, 0.3),
             (1e25, 1e24, 1.1e25)]

@pytest.mark.parametrize('x, y, expected', test_data)
def test_a(x, y, expected):
  sum_ = x + y
  assert sum_ == approx(expected)

Writing test_some_failures.py


In [ ]:
!pytest test_some_failures.py -q

.F...                                                                    [100%]
=================================== FAILURES ===================================
_________________________ test_a[1e+25-1e+23-1.1e+25] __________________________

x = 1e+25, y = 1e+23, expected = 1.1e+25

    @pytest.mark.parametrize('x, y, expected', test_data)
    def test_a(x, y, expected):
      sum_ = x + y
>     assert sum_ == approx(expected)
E     assert 1.01e+25 == 1.1e+25 +- 1.1e+19
E      +  where 1.1e+25 +- 1.1e+19 = approx(1.1e+25)

test_some_failures.py:14: AssertionError
1 failed, 4 passed in 0.04 seconds


In [ ]:
!pytest test_some_failures.py --lf -l

============================= test session starts ==============================
platform linux2 -- Python 2.7.17, pytest-3.6.4, py-1.8.0, pluggy-0.7.1
rootdir: /content, inifile:
collected 5 items / 4 deselected                                               
run-last-failure: rerun previous 1 failure

test_some_failures.py F                                                  [100%]

=================================== FAILURES ===================================
_________________________ test_a[1e+25-1e+23-1.1e+25] __________________________

x = 1e+25, y = 1e+23, expected = 1.1e+25

    @pytest.mark.parametrize('x, y, expected', test_data)
    def test_a(x, y, expected):
      sum_ = x + y
>     assert sum_ == approx(expected)
E     assert 1.01e+25 == 1.1e+25 +- 1.1e+19
E      +  where 1.1e+25 +- 1.1e+19 = approx(1.1e+25)

expected   = 1.1e+25
sum_       = 1.01e+25
x          = 1e+25
y          = 1e+23

test_some_failures.py:14: AssertionError
==================== 1 failed, 4 deselecte

In [ ]:
%%writefile test_slower.py
import datetime
import random
import time

import pytest

@pytest.fixture(autouse=True)
def check_duration(request, cache):
  key = 'duration/' + request.node.nodeid.replace(':', '_')
  start_time = datetime.datetime.now()
  yield
  stop_time = datetime.datetime.now()
  this_duration = (stop_time - start_time).total_seconds()
  last_duration = cache.get(key, None)

  cache.set(key, this_duration)
  if last_duration is not None:
    errorstring = "test duration over 2x last duration"
    assert this_duration <= last_duration * 2, errorstring


@pytest.mark.parametrize('i', range(5))
def test_slow_stuff(i):
  time.sleep(random.random())

Writing test_slower.py


In [ ]:
!pytest test_slower.py -q --tb=line

.....                                                                    [100%]
5 passed in 1.52 seconds


In [ ]:
!pytest -q --cache-show

cachedir: /content/.pytest_cache
--------------------------------- cache values ---------------------------------
cache/lastfailed contains:
  {u'test_pass_fail.py::test_that_fails': True,
   u'test_some_failures.py::test_a[1e+25-1e+23-1.1e+25]': True,
   u'test_tmpdir.py::test_tmpdir': True}
cache/nodeids contains:
  [u'test_slower.py::test_slow_stuff[0]',
   u'test_slower.py::test_slow_stuff[1]',
   u'test_slower.py::test_slow_stuff[2]',
   u'test_slower.py::test_slow_stuff[3]',
   u'test_slower.py::test_slow_stuff[4]']
duration/test_slower.py__test_slow_stuff[0] contains:
  0.113925
duration/test_slower.py__test_slow_stuff[1] contains:
  0.168398
duration/test_slower.py__test_slow_stuff[2] contains:
  0.060191
duration/test_slower.py__test_slow_stuff[3] contains:
  0.646472
duration/test_slower.py__test_slow_stuff[4] contains:
  0.511017

no tests ran in 0.00 seconds


In [ ]:
%%writefile test_slower2.py
from collections import namedtuple
import datetime
import random
import time

import pytest

Duration = namedtuple('Duration', ['current', 'last'])


@pytest.fixture(scope='session')
def duration_cache(request):
  key = 'duration/testdurations'
  d = Duration({}, request.config.cache.get(key, {}))
  yield d
  request.config.cache.set(key, d.current)

@pytest.fixture(autouse=True)
def check_duration(request, duration_cache):
  d = duration_cache
  nodeid = request.node.nodeid
  start_time = datetime.datetime.now()
  yield
  stop_time = datetime.datetime.now()
  duration = (stop_time - start_time).total_seconds()
  d.current[nodeid] = duration

  if d.last.get(nodeid, None) is not None:
    errorstring = "test duration over 2x last duration"
    assert duration <= d.last[nodeid]* 2, errorstring


@pytest.mark.parametrize('i', range(5))
def test_slow_stuff(i):
  time.sleep(random.random())

Writing test_slower2.py


In [ ]:
!pytest --cache-clear test_slower2.py

============================= test session starts ==============================
platform linux2 -- Python 2.7.17, pytest-3.6.4, py-1.8.0, pluggy-0.7.1
rootdir: /content, inifile:
collected 5 items                                                              

test_slower2.py .....                                                    [100%]

=========================== 5 passed in 1.55 seconds ===========================


In [ ]:
!pytest test_slower2.py

============================= test session starts ==============================
platform linux2 -- Python 2.7.17, pytest-3.6.4, py-1.8.0, pluggy-0.7.1
rootdir: /content, inifile:
collected 5 items                                                              

test_slower2.py .E...E.E                                                 [100%]

==================================== ERRORS ====================================
___________________ ERROR at teardown of test_slow_stuff[0] ____________________

request = <SubRequest 'check_duration' for <Function 'test_slow_stuff[0]'>>
duration_cache = Duration(current={'test_slower2.py::test_slow_stuff[0]': 0.89221}, last={u'tes...st_slow_stuff[1]': 0.621141, u'test_slower2.py::test_slow_stuff[0]': 0.387305})

    @pytest.fixture(autouse=True)
    def check_duration(request, duration_cache):
      d = duration_cache
      nodeid = request.node.nodeid
      start_time = datetime.datetime.now()
      yield
      stop_time = datetime.datetime.now()


In [ ]:
!pytest --cache-show

============================= test session starts ==============================
platform linux2 -- Python 2.7.17, pytest-3.6.4, py-1.8.0, pluggy-0.7.1
rootdir: /content, inifile:
cachedir: /content/.pytest_cache
--------------------------------- cache values ---------------------------------
cache/lastfailed contains:
  {u'test_slower2.py::test_slow_stuff[0]': True,
   u'test_slower2.py::test_slow_stuff[3]': True,
   u'test_slower2.py::test_slow_stuff[4]': True}
cache/nodeids contains:
  [u'test_slower2.py::test_slow_stuff[0]',
   u'test_slower2.py::test_slow_stuff[1]',
   u'test_slower2.py::test_slow_stuff[2]',
   u'test_slower2.py::test_slow_stuff[3]',
   u'test_slower2.py::test_slow_stuff[4]']
duration/testdurations contains:
  {u'test_slower2.py::test_slow_stuff[0]': 0.89221,
   u'test_slower2.py::test_slow_stuff[1]': 0.131511,
   u'test_slower2.py::test_slow_stuff[2]': 0.821144,
   u'test_slower2.py::test_slow_stuff[3]': 0.358413,
   u'test_slower2.py::test_slow_stuff[4]': 0.8693

# Using capsys

capsys allows you to retrieve stdout and stderr from some code, and it disables output capture temporarily. 

In [ ]:
%%writefile test_capsys.py

def greeting(name):
  print('Hi, %s'%name)

def test_greeting(capsys):
  greeting('Earthling')
  out, err = capsys.readouterr()
  assert out == 'Hi, Earthling\n'
  assert err == ""

  greeting('Brian')
  greeting('Ned')
  out, err = capsys.readouterr()

  assert out == 'Hi, Brian\nHi, Ned\n'
  assert err == ''

Writing test_capsys.py


In [ ]:
!pytest test_capsys.py -v

============================= test session starts ==============================
platform linux2 -- Python 2.7.17, pytest-3.6.4, py-1.8.0, pluggy-0.7.1 -- /usr/bin/python2
cachedir: .pytest_cache
rootdir: /content, inifile:
collected 1 item                                                               

test_capsys.py::test_greeting PASSED                                     [100%]

=========================== 1 passed in 0.01 seconds ===========================


In [ ]:
%%writefile test_capsys2.py
import sys

def yikes(problem):
  print('Yikes! %s'%problem, file=sys.stderr)


def test_yikes(capsys):
  yikes('Out of batterie!')
  out, err = capsys.readouterr()

  assert out == ""
  assert 'Out of batteries!' in err

Writing test_capsys2.py


In [ ]:
!python3 -m pytest test_capsys2.py -s

============================= test session starts ==============================
platform linux -- Python 3.7.11, pytest-3.6.4, py-1.10.0, pluggy-0.7.1
rootdir: /content, inifile:
plugins: typeguard-2.7.1
collected 1 item                                                               

test_capsys2.py F

=================================== FAILURES ===================================
__________________________________ test_yikes __________________________________

capsys = <_pytest.capture.CaptureFixture object at 0x7f9d9eb22dd0>

    def test_yikes(capsys):
      yikes('Out of batterie!')
      out, err = capsys.readouterr()
    
      assert out == ""
>     assert 'Out of batteries!' in err
E     AssertionError: assert 'Out of batteries!' in 'Yikes! Out of batterie!\n'

test_capsys2.py:12: AssertionError
=========================== 1 failed in 0.03 seconds ===========================


# Print without pytest capturing

In [ ]:
%%writefile test_capsys3.py
def test_capsys_disabled(capsys):
  with capsys.disabled():
    print('\nalways print this')
  print('normal print, captured!')

Writing test_capsys3.py


In [ ]:
!pytest test_capsys3.py -s

============================= test session starts ==============================
platform linux2 -- Python 2.7.17, pytest-3.6.4, py-1.8.0, pluggy-0.7.1
rootdir: /content, inifile:
collected 1 item                                                               

test_capsys3.py 
always print this
normal print, captured!
.

=========================== 1 passed in 0.01 seconds ===========================


In [ ]:
!pytest test_capsys3.py

============================= test session starts ==============================
platform linux2 -- Python 2.7.17, pytest-3.6.4, py-1.8.0, pluggy-0.7.1
rootdir: /content, inifile:
collected 1 item                                                               

test_capsys3.py 
always print this
.                                                        [100%]

=========================== 1 passed in 0.01 seconds ===========================


# Using monkeypatch

In [ ]:
import os
import json


def read_cheese_preferences():
  full_path = os.path.expanduser('~/.cheese.json')
  with open(full_path) as f:
    prefs = json.load(f)
  return prefs

def write_cheese_preferences(prefs):
  full_path = os.path.expanduser('~/.cheese.json')
  with open(full_path, 'w') as f:
    json.dumps(prefs, f, indent=4)


def write_default_cheese_preferences():
  write_cheese_preferences(_default_prefs)

_default_prefs = {
    'slicing': ['manchego', 'sharp cheddar'],
    'spreadable': ['Saint Andre', 'camembert'],
    'salads': ['crumbled feta']
}

def test_def_prefs_change_home(tmpdir, monkeypatch):
  fake_home_dir = tmpdir.mkdir('home')
  monkeypatch.setattr(cheese.os.path, 'expanduser',
                      lambda x: x.replace('~', str(fake_home_dir)))
  
  cheese.write_default_cheese_preferences()
  defaults_before = copy.deepcopy(cheese._default_prefs)

  # change the defaults
  monkeypatch.setitem(cheese._default_prefs, 'slicing', ['provolone'])
  monkeypatch.setitem(cheese._default_prefs, 'spreadable', ['brie'])
  monkeypatch.setitem(cheese._default_prefs, 'salads', ['pepper jack'])
  defaults_modified = cheese._default_prefs
  
  cheese.write_default_cheese_preferences()

  actual = cheese.read_cheese_preferences()
  assert defaults_modified == actual
  assert defaults_modified != defaults_before

syspath_prepend(path): Adds a new path to your list of module import directories. This can be used to replace a system-wide module or package with a fake version.


chdir(path): change the current working directory. You could change to a temporary directory that holds whatever contents you need for a command-line script or something

monkeypatch fixtures also work in conjunction with unittest.mock to temporarily replace attributes with mock objects.

# Using doctest_namespace

In [ ]:
%%writefile my_math.py
"""
This file defines multiply(a, b) and divide(a, b).

>>> import my_math as mm

Here's how you use multiply:

>>> mm.multiply(4, 3)
12

>>> mm.multiply('a', 3)
'aaa'


>>> mm.divide(10, 5)
2.0

"""

def multiply(a, b):
  """
  Returns a * b.

  >>> mm.multiply(4, 3)
  12

  >>> mm.multiply('a', 3)
  'aaa'
  """
  return a*b

def divide(a, b):
  """
  Returns a / b

  >>> mm.divide(10, 5)
  2.0
  """
  return a / b

Overwriting my_math.py


In [ ]:
!python3 -m pytest -v --doctest-modules --tb=short my_math.py

============================= test session starts ==============================
platform linux -- Python 3.7.11, pytest-3.6.4, py-1.10.0, pluggy-0.7.1 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content, inifile:
plugins: typeguard-2.7.1
collected 3 items                                                              

my_math.py::my_math PASSED                                               [ 33%]
my_math.py::my_math.divide FAILED                                        [ 66%]
my_math.py::my_math.multiply FAILED                                      [100%]

=================================== FAILURES ===================================
___________________________ [doctest] my_math.divide ___________________________
033 
034   Returns a / b
035 
036   >>> mm.divide(10, 5)
UNEXPECTED EXCEPTION: NameError("name 'mm' is not defined")
Traceback (most recent call last):

  File "/usr/lib/python3.7/doctest.py", line 1337, in __run
    compileflags, 1), test.globs)

  File "<doctest my

In [ ]:
%%writefile conftest.py
import pytest

import my_math

@pytest.fixture(autouse=True)
def add_mm(doctest_namespace):
  doctest_namespace['mm'] = my_math

Overwriting conftest.py


In [ ]:
!python3 -m pytest -v --doctest-modules --tb=short my_math.py

============================= test session starts ==============================
platform linux -- Python 3.7.11, pytest-3.6.4, py-1.10.0, pluggy-0.7.1 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content, inifile:
plugins: typeguard-2.7.1
collected 3 items                                                              

my_math.py::my_math PASSED                                               [ 33%]
my_math.py::my_math.divide PASSED                                        [ 66%]
my_math.py::my_math.multiply PASSED                                      [100%]

=========================== 3 passed in 0.01 seconds ===========================


# Using recwarn

In [ ]:
%%writefile test_warnings.py
import warnings
import pytest


def bad_function():
  warnings.warn('This is bad', DeprecationWarning)


def test_bad_function(recwarn):
  bad_function()
  assert len(recwarn) == 1

  w = recwarn.pop()
  assert w.category == DeprecationWarning
  assert str(w.message) == 'This is bad'

Overwriting test_warnings.py


In [ ]:
!pytest test_warnings.py

============================= test session starts ==============================
platform linux2 -- Python 2.7.17, pytest-3.6.4, py-1.8.0, pluggy-0.7.1
rootdir: /content, inifile:
collected 1 item                                                               

test_warnings.py .                                                       [100%]

=========================== 1 passed in 0.01 seconds ===========================


In [ ]:
%%writefile test_warnings2.py
import warnings
import pytest


def bad_function():
  warnings.warn('This is bad', DeprecationWarning)

def test_bad_function2():
  with pytest.warns(None) as warning_list:
    bad_function()

  assert len(warning_list) == 1

  w = warning_list.pop()
  assert w.category == DeprecationWarning
  assert str(w.message) == 'This is bad'

Overwriting test_warnings2.py


In [ ]:
!pytest test_warnings2.py -v

============================= test session starts ==============================
platform linux2 -- Python 2.7.17, pytest-3.6.4, py-1.8.0, pluggy-0.7.1 -- /usr/bin/python2
cachedir: .pytest_cache
rootdir: /content, inifile:
collected 1 item                                                               

test_warnings2.py::test_bad_function2 PASSED                             [100%]

=========================== 1 passed in 0.01 seconds ===========================
